In [1]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index import *

ModuleNotFoundError: No module named 'google'

### Tokenizer
Problems:
- its doesn't catch words the same if they have signs words != words?


In [ ]:
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stopwords_frozen = frozenset(stopwords.words('english'))
def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filter stopwords.
    
    Parameters:
    -----------
    text: string , represting the text to tokenize.    
    
    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in stopwords_frozen]    
    return list_of_tokens

default_tokenizer = lambda text: tokenize(text)

In [3]:
import json
from itertools import chain

with open('queries_train.json') as f:
    data = json.load(f)
# (q_id : (query_text, [relvent docs_ids]))
test_data = {q_id: d for q_id, d in enumerate(data.items())}
queries = {q_id: qf[0] for q_id, qf in test_data.items()}
queries_tokenize = {q_id: tokenize(q) for q_id, q in queries.items()}


## Replace temporary solution with replace, to eliminate signs words != words? BETTER PARSER
queries_words = set()
for l in queries.values():
  for w in l.split():
    queries_words.add(w.replace('?', ''))

queries_words



{'2022',
 'Air',
 'Apple',
 'Ciggarets',
 'Cup',
 'Dolly',
 'Elon',
 'How',
 'India',
 'Information',
 'Java',
 'Jordan',
 'Language',
 'LinkedIn',
 'Marijuana',
 'Morty',
 'Most',
 'Natural',
 'Netflix',
 'Rick',
 'Ritalin',
 'Simpsons',
 'The',
 'What',
 'Winter',
 'World',
 'and',
 'at',
 'best',
 'breed',
 'city',
 'coffee',
 'come',
 'computer',
 'cup',
 'deal',
 'depression',
 'do',
 'expensive',
 'fast',
 'flowers',
 'gold',
 'home',
 'how',
 'hummus',
 'in',
 'is',
 'kids',
 'live',
 'lose',
 'make',
 'marvel',
 'money',
 'movie',
 'musk',
 'place',
 'processing',
 'retrieval',
 'sheep',
 'the',
 'to',
 'weight',
 'wine',
 'with',
 'world',
 'you'}

In [4]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.9 MB/s eta 0:00:00
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 124016 files and directories currently installed.)
Pre

In [5]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [6]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [7]:
spark

In [8]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except: 
  pass

In [9]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'main-analog-370510'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 


Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [10]:
from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [11]:
doc_body_pairs = parquetFile.limit(1).select("id", "text")
doc_title_pairs = parquetFile.limit(1).select("id", "title")
doc_anchor_pairs = parquetFile.limit(1).select("id", "anchor_text")

In [12]:
doc_title_pairs.printSchema()
doc_body_pairs.printSchema()
doc_anchor_pairs.show()

root
 |-- id: long (nullable = true)
 |-- title: string (nullable = true)

root
 |-- id: long (nullable = true)
 |-- text: string (nullable = true)

+---+--------------------+
| id|         anchor_text|
+---+--------------------+
| 12|[{23040, politica...|
+---+--------------------+



In [13]:
doc_body_pairs_rdd = parquetFile.limit(1000).select("id", "text").rdd
doc_title_pairs_rdd = parquetFile.limit(1000).select("id", "title").rdd
doc_anchor_pairs_rdd = parquetFile.limit(1000).select("id", "anchor_text").rdd

### Tokenizer
will change in the future do classify:
- Bold
- Dates
- U.S.A == USA


#### Reset the file system

In [14]:

def process_wiki(pages, index_name='first', tokenize_func=default_tokenizer, words=None):
  """ Process wikipedia: tokenize article body, title, anchor text, and create
      indices for them. Each index is named `index_name` and placed in a 
      directory under the current dir named 'body_indices', 'title_index' 
      and 'anchor_index', respectively. 
  Parameters:
  -----------
  pages: list of tuples
    Each tuple is a wiki article with id, title, body, and 
    [(target_article_id, anchor_text), ...]. 
  index_name: str
    The name for the index.
  tokenize_func: function str -> list of str
    Tokenization function that takes text as input and return a list of 
    tokens.
  Returns:
  --------
  Three inverted index objects
    body_index, title_index, anchor_index.
  """
  # create the index for titles
  # collect anchor text tokens for each target article by its id

  # iterate over batches of pages from the dump
  # tokenize
  # create a separate index of articles body for article in this batch
  id2anchor_text = defaultdict(list)
  body_index = InvertedIndex(data_set=words)
  title_index = InvertedIndex(data_set=words)
  anchor_index = InvertedIndex(data_set=words)
  batch_idx, documents = pages

  for id, title, body, anchor in documents:
    body_index.add_doc(id, tokenize_func(body))
    title_index.add_doc(id, tokenize_func(title))

    for target_id, anchor_text in anchor:
      anchor_index.add_doc(target_id, anchor_text)


  body_index.write('./body_indices', f'{index_name}_{batch_idx}')
  title_index.write('./title_index', f'{index_name}_{batch_idx}')
  anchor_index.write('./anchor_index', f'{index_name}_{batch_idx}')

  return f'{index_name}_{batch_idx}'


In [ ]:
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS
bucket_names = parquetFile.rdd.map(lambda document: (token2bucket_id(str(document[0])), list(document))).groupByKey().map(lambda pages: process_wiki(pages, words=queries_words)).collect()

In [ ]:
import shutil
shutil.rmtree('./body_indices', ignore_errors=True)
shutil.rmtree('./title_index', ignore_errors=True)
shutil.rmtree('./anchor_index', ignore_errors=True)

!mkdir body_indices title_index anchor_index


In [ ]:
# doc_body_pairs_rdd.map(lambda page: process_wiki(page, index_name='first', index_part = 'body_indices', tokenize_func=default_tokenizer))
# doc_anchor_pairs_rdd.map(lambda page: process_wiki(page, index_name='first', index_part = 'anchor_index', tokenize_func=default_tokenizer))
# doc_title_pairs_rdd.map(lambda page: process_wiki(page, index_name='first', index_part = 'title_index', tokenize_func=default_tokenizer))

### Process wiki
Here we process the wikipedia pages given. working on batchs sended (RDD) format (id, title, body, anchor)
we build indexs for each batch:
- title_index
- text_index
- anchor index
and return the as list RDD contain 
f'{
  index_name= name how you like,
  _
  batch_idx= bucket index
}
we hold all data in inverted index:
- posting_lists


In [ ]:
with open('buckets.txt', 'w') as f:
    for bucket in bucket_names:
        f.write(f"{bucket}\n")

In [ ]:
with open("buckets", "rb") as fp:
  bucket_names = pickle.load(fp)

In [ ]:
title_idx = InvertedIndex()
title_idx.merge_indices('title_index', bucket_names, 'first')


In [ ]:
body_idx = InvertedIndex()
body_idx.merge_indices('body_indices', bucket_names, 'first')

In [ ]:
anchor_idx = InvertedIndex()
body_idx.merge_indices('title_index', bucket_names, 'first')

In [ ]:
# from search import Search
# from cosineSimilarity import Cosine_Similarity

# queries_tokenize = dict(list({q_id: tokenize(q) for q_id, q in queries.items()}.items())[0])
# search = Search()
# tfidf_queries_score_train_cosS_body = search.get_topN_score_for_queries(queries_tokenize, body_idx, N=30, score=Cosine_Similarity)

In [ ]:
from urllib.request import urlopen
import json

def get_page_titles(pagesId):
  ''' Returns the title of the first, fourth, and fifth pages as ranked about 
      by PageRank.
  Returns:
  --------
    list of three strings.
  '''
  # YOUR CODE HERE
  def get_pageId_title(pageID):
    url_json_wiki = lambda x : f"https://en.wikipedia.org/w/api.php?action=query&inprop=url&pageids={x}&prop=info&format=json"   
    # store the response of URL
    response = urlopen(url_json_wiki(pageID)) 
    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())
    try:
      return data_json['query']['pages'][str(pageID)]['title']
    except:
      return pageID
    

  titles = [get_pageId_title(pageId) for pageId in pagesId]
  return titles

In [ ]:
from search import Search
from cosineSimilarity import Cosine_Similarity
from binarySimilarity import Binary_Similarity

queries_tokenize = {q_id: tokenize(q) for q_id, q in queries.items()}
search = Search()
tfidf_queries_score_train_bsS_title = search.get_topN_score_for_queries(queries_tokenize, title_idx, N=30, score=Binary_Similarity)
# tfidf_queries_score_train_bsS_anchor = [search.get_topN_score_for_queries({q_id: query}, anchor_idx, N=30, score=Binary_Similarity) for q_id, query in queries_tokenize.items()]


In [ ]:
queries_results = {queries[q_id] : list(map(get_page_titles, results)) for q_id, results in tfidf_queries_score_train_bsS_title.items()}
queries_results